# Tutorial: Criando uma API REST com Django REST Framework - Lista de Tarefas

Este tutorial ensina como criar uma API REST completa para gerenciamento de tarefas usando Django e Django REST Framework.

## Pré-requisitos
- Python 3.8+ instalado
- Conhecimento básico de Django
- Conhecimento básico de APIs REST

## Passo 1: Configuração Inicial do Projeto

### 1.1 Criando o ambiente virtual
Execute os comandos abaixo no terminal:

```bash
python -m venv venv
source venv/bin/activate  # No macOS/Linux
# venv\Scripts\activate   # No Windows
```

### 1.2 Instalando as dependências

Crie um arquivo `requirements.txt` com o seguinte conteúdo:

In [ ]:
# Conteúdo do requirements.txt
requirements_content = """
Django
djangorestframework
"""

# Salvar o arquivo
with open('requirements.txt', 'w') as f:
    f.write(requirements_content.strip())

print("Arquivo requirements.txt criado!")
print("Execute no terminal: pip install -r requirements.txt")

### 1.3 Criando o projeto Django

Execute no terminal:
```bash
django-admin startproject lista_tarefas
cd lista_tarefas
```

### 1.4 Criando o app API
```bash
python manage.py startapp api
```

## Passo 2: Configuração do Settings

Edite o arquivo `lista_tarefas/settings.py` e adicione as aplicações necessárias:

In [ ]:
# Exemplo da configuração INSTALLED_APPS no settings.py
INSTALLED_APPS_example = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'api',  # Nova app
    'rest_framework'  # Django REST Framework
]

print("Adicione 'api' e 'rest_framework' ao INSTALLED_APPS no settings.py")
for app in INSTALLED_APPS_example:
    print(f"    '{app}',")

## Passo 3: Criando o Modelo de Dados

No arquivo `api/models.py`, crie o modelo `Tarefa`:

In [ ]:
# Conteúdo do arquivo api/models.py
models_content = '''
from django.db import models

class Tarefa(models.Model):
    status_choice = {
        'P': 'PENDENTE',
        'A': 'ANDAMENTO',
        'C': 'CONCLUIDO'
    }

    titulo = models.CharField(max_length=200)
    descricao = models.TextField(blank=True)
    status = models.CharField(max_length=1, choices=status_choice, default='P')
    data_criacao = models.DateField(auto_now=True)

    def __str__(self):
        return self.titulo
'''

print("Modelo Tarefa:")
print(models_content)

### Explicação do Modelo:

- **titulo**: Campo de texto para o título da tarefa (máximo 200 caracteres)
- **descricao**: Campo de texto longo opcional para descrição
- **status**: Campo de escolha com três opções (Pendente, Andamento, Concluído)
- **data_criacao**: Campo de data que é automaticamente preenchido

## Passo 4: Criando e Aplicando Migrações

Execute no terminal:
```bash
python manage.py makemigrations
python manage.py migrate
```

## Passo 5: Criando o Serializer

No arquivo `api/serializers.py`, crie o serializer:

In [ ]:
# Conteúdo do arquivo api/serializers.py
serializers_content = '''
from rest_framework import serializers
from .models import Tarefa

class TarefaSerializer(serializers.ModelSerializer):
    class Meta:
        model = Tarefa
        fields = ['titulo', 'descricao', 'status', 'data_criacao']
        read_only_fields = ['data_criacao']
'''

print("Serializer:")
print(serializers_content)

print("\n### Explicação do Serializer:")
print("- Converte objetos Python em formato JSON e vice-versa")
print("- ModelSerializer automatiza muito do processo")
print("- read_only_fields impede que o campo seja modificado via API")

## Passo 6: Criando as Views da API

No arquivo `api/views.py`, implemente todas as views:

In [ ]:
# Conteúdo do arquivo api/views.py - Parte 1: Imports e lista_tarefas
views_part1 = '''
from rest_framework.decorators import api_view
from rest_framework.response import Response 
from rest_framework import status

from .models import Tarefa
from .serializers import TarefaSerializer

@api_view(['GET'])
def lista_tarefas(request):
    if request.method == 'GET':
        tarefas = Tarefa.objects.all().order_by('status')
        tarefa_serializer = TarefaSerializer(tarefas, many=True)
        return Response(tarefa_serializer.data)
'''

print("Views - Parte 1 (Imports e lista_tarefas):")
print(views_part1)

In [ ]:
# Conteúdo do arquivo api/views.py - Parte 2: lista_concluidas e cria_tarefa
views_part2 = '''
@api_view(['GET'])
def lista_concluidas(request):
    if request.method == 'GET':
        tarefas_concluidas = Tarefa.objects.filter(status='C')
        tarefa_serializer = TarefaSerializer(tarefas_concluidas, many=True)
        return Response(tarefa_serializer.data)

@api_view(['POST'])
def cria_tarefa(request):
    if request.method == 'POST':
        tarefa_serializer = TarefaSerializer(data=request.data)
        
        if tarefa_serializer.is_valid():
            tarefa_serializer.save()
            return Response(tarefa_serializer.data, status=status.HTTP_201_CREATED)
    
    return Response(tarefa_serializer.errors, status=status.HTTP_400_BAD_REQUEST)
'''

print("Views - Parte 2 (lista_concluidas e cria_tarefa):")
print(views_part2)

In [ ]:
# Conteúdo do arquivo api/views.py - Parte 3: atualiza_tarefa
views_part3 = '''
@api_view(['PUT', 'PATCH'])
def atualiza_tarefa(request, pk):
    try:
        tarefa = Tarefa.objects.get(pk=pk)
    except Tarefa.DoesNotExist:
        return Response({"error": "Tarefa não encontrada"}, status=status.HTTP_404_NOT_FOUND)

    if request.method == 'PUT' or request.method == 'PATCH':
        atualizar_parcial = True if request.method == 'PATCH' else False
        
        tarefa_serializer = TarefaSerializer(tarefa, data=request.data, partial=atualizar_parcial)
        
        if tarefa_serializer.is_valid():
            tarefa_serializer.save()
            return Response(tarefa_serializer.data, status=status.HTTP_200_OK)
        
        return Response(tarefa_serializer.errors, status=status.HTTP_400_BAD_REQUEST)
'''

print("Views - Parte 3 (atualiza_tarefa):")
print(views_part3)

In [ ]:
# Conteúdo do arquivo api/views.py - Parte 4: deleta_tarefa
views_part4 = '''
@api_view(['DELETE'])
def deleta_tarefa(request, pk):
    try:
        tarefa = Tarefa.objects.get(pk=pk)
    except Tarefa.DoesNotExist:
        return Response({"error": "Tarefa não encontrada"}, status=status.HTTP_404_NOT_FOUND)

    if request.method == 'DELETE':
        tarefa.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)
'''

print("Views - Parte 4 (deleta_tarefa):")
print(views_part4)

### Explicação das Views:

1. **lista_tarefas**: Lista todas as tarefas ordenadas por status
2. **lista_concluidas**: Lista apenas tarefas concluídas (status 'C')
3. **cria_tarefa**: Cria nova tarefa via POST
4. **atualiza_tarefa**: Atualiza tarefa existente (PUT = completo, PATCH = parcial)
5. **deleta_tarefa**: Remove tarefa pelo ID

## Passo 7: Configurando as URLs

### 7.1 URLs do App (`api/urls.py`)

In [ ]:
# Conteúdo do arquivo api/urls.py
api_urls_content = '''
from django.urls import path
from .views import lista_tarefas, cria_tarefa, atualiza_tarefa, deleta_tarefa, lista_concluidas

urlpatterns = [
    path('', lista_tarefas, name='lista_tarefas'),
    path('criar/', cria_tarefa, name='cria_tarefa'),
    path('atualiza/<int:pk>/', atualiza_tarefa, name='atualiza_tarefa'),
    path('deleta/<int:pk>/', deleta_tarefa, name='deleta_tarefa'),
    path('concluidas/', lista_concluidas, name='lista_concluidas')
]
'''

print("URLs do App (api/urls.py):")
print(api_urls_content)

In [ ]:
# Conteúdo do arquivo lista_tarefas/urls.py
project_urls_content = '''
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('api.urls')),
]
'''

print("URLs do Projeto (lista_tarefas/urls.py):")
print(project_urls_content)

## Passo 8: Testando a API

### 8.1 Iniciar o servidor
```bash
python manage.py runserver
```

### 8.2 Endpoints disponíveis:

In [ ]:
# Mapeamento dos endpoints da API
endpoints = {
    "GET /": "Lista todas as tarefas",
    "GET /concluidas/": "Lista tarefas concluídas",
    "POST /criar/": "Cria nova tarefa",
    "PUT/PATCH /atualiza/{id}/": "Atualiza tarefa",
    "DELETE /deleta/{id}/": "Remove tarefa"
}

print("Endpoints da API:")
print("=" * 40)
for endpoint, description in endpoints.items():
    print(f"{endpoint:<25} - {description}")

### 8.3 Exemplos de uso com cURL:

In [ ]:
# Exemplos de comandos cURL para testar a API
curl_examples = {
    "Criar tarefa": '''
curl -X POST http://127.0.0.1:8000/criar/ \
  -H "Content-Type: application/json" \
  -d '{"titulo": "Minha primeira tarefa", "descricao": "Descrição da tarefa"}'
''',
    "Listar tarefas": '''
curl http://127.0.0.1:8000/
''',
    "Atualizar status": '''
curl -X PATCH http://127.0.0.1:8000/atualiza/1/ \
  -H "Content-Type: application/json" \
  -d '{"status": "C"}'
''',
    "Deletar tarefa": '''
curl -X DELETE http://127.0.0.1:8000/deleta/1/
'''
}

for action, command in curl_examples.items():
    print(f"**{action}:**")
    print(f"```bash{command}```\n")

## Passo 9: Melhorias Opcionais

### 9.1 Registrar no Django Admin (`api/admin.py`)

In [ ]:
# Conteúdo do arquivo api/admin.py
admin_content = '''
from django.contrib import admin
from .models import Tarefa

@admin.register(Tarefa)
class TarefaAdmin(admin.ModelAdmin):
    list_display = ['titulo', 'status', 'data_criacao']
    list_filter = ['status', 'data_criacao']
    search_fields = ['titulo']
'''

print("Admin (api/admin.py):")
print(admin_content)

print("\n### Para acessar o admin:")
print("1. Criar superusuário: python manage.py createsuperuser")
print("2. Acessar: http://127.0.0.1:8000/admin/")

## Testando a API com Python

In [ ]:
# Exemplo de como testar a API usando Python requests
# (Este código funcionará quando o servidor Django estiver rodando)

import requests
import json

# URL base da API
BASE_URL = "http://127.0.0.1:8000"

def test_api():
    """Função para testar todos os endpoints da API"""
    
    print("=== TESTANDO API DE TAREFAS ===")
    
    # 1. Criar uma nova tarefa
    print("\n1. Criando nova tarefa...")
    nova_tarefa = {
        "titulo": "Tarefa de Teste",
        "descricao": "Esta é uma tarefa criada para teste",
        "status": "P"
    }
    
    try:
        response = requests.post(f"{BASE_URL}/criar/", json=nova_tarefa)
        if response.status_code == 201:
            tarefa_criada = response.json()
            print(f"✅ Tarefa criada: {tarefa_criada}")
            tarefa_id = tarefa_criada.get('id')
        else:
            print(f"❌ Erro ao criar tarefa: {response.status_code}")
            return
    except requests.exceptions.ConnectionError:
        print("❌ Servidor Django não está rodando. Execute 'python manage.py runserver'")
        return
    
    # 2. Listar todas as tarefas
    print("\n2. Listando todas as tarefas...")
    response = requests.get(f"{BASE_URL}/")
    if response.status_code == 200:
        tarefas = response.json()
        print(f"✅ Total de tarefas: {len(tarefas)}")
        for tarefa in tarefas:
            print(f"   - {tarefa['titulo']} (Status: {tarefa['status']})")
    
    # 3. Atualizar status da tarefa
    if 'tarefa_id' in locals():
        print(f"\n3. Atualizando status da tarefa {tarefa_id}...")
        update_data = {"status": "C"}
        response = requests.patch(f"{BASE_URL}/atualiza/{tarefa_id}/", json=update_data)
        if response.status_code == 200:
            print("✅ Status atualizado para 'CONCLUÍDO'")
    
    # 4. Listar tarefas concluídas
    print("\n4. Listando tarefas concluídas...")
    response = requests.get(f"{BASE_URL}/concluidas/")
    if response.status_code == 200:
        tarefas_concluidas = response.json()
        print(f"✅ Tarefas concluídas: {len(tarefas_concluidas)}")

# Para executar o teste, descomente a linha abaixo
# test_api()

print("Função de teste criada!")
print("Para testar, certifique-se de que o servidor Django está rodando e execute: test_api()")

## Conceitos Aprendidos

Ao final deste tutorial, você aprendeu:

In [ ]:
# Resumo dos conceitos aprendidos
conceitos = {
    "Models": "Definição da estrutura de dados no Django",
    "Serializers": "Conversão entre objetos Python e formato JSON",
    "Views": "Lógica de negócio da API (function-based views)",
    "URLs": "Roteamento de endpoints da API",
    "HTTP Methods": "GET, POST, PUT, PATCH, DELETE",
    "Status Codes": "200 (OK), 201 (Created), 400 (Bad Request), 404 (Not Found), etc.",
    "Validação": "Validação automática através de serializers",
    "Filtros": "Query de dados específicos usando Django ORM",
    "CRUD": "Create, Read, Update, Delete - operações básicas",
    "API REST": "Princípios de design de APIs RESTful"
}

print("CONCEITOS APRENDIDOS:")
print("=" * 50)

for i, (conceito, descricao) in enumerate(conceitos.items(), 1):
    print(f"{i:2d}. {conceito:<15} - {descricao}")

print("\n🎉 Parabéns! Você criou uma API REST completa com Django REST Framework!")

## Próximos Passos

Para continuar aprendendo, considere:

1. **Autenticação e Autorização**: Adicionar login e permissões
2. **Paginação**: Para listar muitos dados de forma eficiente
3. **Filtros Avançados**: Usando django-filter
4. **Documentação**: Usando drf-spectacular para Swagger/OpenAPI
5. **Testes**: Criar testes automatizados para a API
6. **Deploy**: Colocar a API em produção
7. **Versionamento**: Gerenciar versões da API
8. **Cache**: Otimizar performance com Redis

Esta API implementa um CRUD completo seguindo as melhores práticas do Django REST Framework!